### Octavio Revetria 232745

# Reinforcement Learning Tarea 2 - Programación Dinámica (Grid World)

## Objetivos
- Entender y aplicar los conceptos de Programación Dinámica en Aprendizaje por Refuerzo.
- Implementar los algoritmos de Evaluación de Política (Policy Evaluation), Mejora de Política (Policy Improvement) e Iteración de Valor (Value Iteration).
- Obtener y analizar políticas óptimas en un ambiente de Grid World.

## A entregar
- Implementación del método de Iteración de Política.
- Implementación del método de Iteración de Valor.
- Comparación de los resultados obtenidos y discusión sobre convergencia y estabilidad.

## Descripción del ambiente a usar

Trabajaremos con el ambiente **Grid World** descrito en el capítulo 4 del libro de Sutton y Barto. Se trata de un entorno en forma de cuadrícula donde el agente puede moverse en cuatro direcciones:
- **RIGHT = 0** (derecha)
- **UP = 1** (arriba)
- **LEFT = 2** (izquierda)
- **DOWN = 3** (abajo)

El objetivo del agente es alcanzar uno de los dos estados terminales ubicados en la esquina superior izquierda y la esquina inferior derecha de la cuadrícula. Cada movimiento tiene un costo de -1, incentivando así la búsqueda de la ruta más corta hacia un estado terminal. Si el agente intenta moverse fuera de los límites de la cuadrícula, permanecerá en su posición actual.

Un ejemplo de un Grid World 4x4:
```
T  o  o  o
o  x  o  o
o  o  o  o
o  o  o  T
```
- "T" representa un estado terminal.
- "x" representa la posición actual del agente.
- "o" representa los estados transitables.

La dinámica de la recompensa es simple: el agente recibe un **recompensa de -1** en cada paso hasta alcanzar un estado terminal.

> Quizás se necesita:
> https://anaconda.org/conda-forge/gymnasium-box2d
> https://anaconda.org/conda-forge/gymnasium-other
> https://ffmpeg.org/download.html (en mac: `brew install ffmpeg`)

In [77]:
import gymnasium as gym
from gymnasium.wrappers import RecordEpisodeStatistics, RecordVideo
import numpy as np
import GridWorldEnv

from Utils import print_state_values, print_state_action_values, print_policy_grids

In [78]:
# Algunas constantes

HUMAN_RENDER = False # False si queremos guardar el episodio en un video, True si queremos verlo en tiempo real (no disponible en Google Colab u otros entornos sin interfaz gráfica)

RECORD_VIDEO = True # True si queremos guardar el episodio en un video, quizas se necesita: https://anaconda.org/conda-forge/gymnasium-other
RECORD_EVERY = 1 # Cada cuántos episodios guardamos un video

GRID_SIZE = 4 # Tamaño del grid
GAMMA = 1 # Factor de descuento

## Gymnasium Wrapper

En Gymnasium, los [wrappers](https://gymnasium.farama.org/api/wrappers/) son herramientas que permiten modificar el comportamiento de los entornos sin alterar su código base. Estos *wrappers* se utilizan para extender funcionalidades, como la normalización de observaciones, limitación de acciones o registro de estadísticas y videos durante el entrenamiento de agentes.

Para este ejercicio, se utilizarán dos *wrappers* de Gymnasium:

**`RecordVideo`**: Este *wrapper* permite grabar videos de las ejecuciones del agente en el entorno. Es especialmente útil para visualizar el comportamiento del agente y evaluar su desempeño. Se puede configurar para grabar todos los episodios o solo episodios específicos, según una función de activación definida por el usuario.

Documentación oficial de Gymnasium sobre [grabación de agentes](https://gymnasium.farama.org/introduction/record_agent/) y la [API de wrappers](https://gymnasium.farama.org/api/wrappers/). 

In [79]:
def get_environment(human_render=HUMAN_RENDER, record_video=RECORD_VIDEO, record_every=RECORD_EVERY, grid_size=GRID_SIZE):
    assert not (human_render and record_video), "No se puede renderizar en tiempo real y guardar un video a la vez"
    
    render_mode = "human" if human_render else "rgb_array"
    
    # Initialise the environment
    env = gym.make("gymnasium_env/GridWorld-v0", render_mode=render_mode, size=grid_size)

    if record_video:
        env = RecordVideo(env, video_folder="./videos", name_prefix="gridworld",
                    episode_trigger=lambda x: x % record_every == 0)
    
    return env

Probamos el entorno y sus wrappers con un agente aleatorio para visualizar el comportamiento del entorno y la grabación de videos.

In [80]:
import os
os.environ["IMAGEIO_FFMPEG_EXE"] = "C:/Users/Octav/anaconda3/envs/ia-taller/Library/bin/ffmpeg.exe"

In [81]:
env = get_environment()

# We play 10 episodes, each one with a random policy
for episode_num in range(10):
    obs, info = env.reset()

    episode_over = False
    while not episode_over:
        action = env.action_space.sample()  # random policy
        obs, reward, terminated, truncated, info = env.step(action)
        episode_over = terminated or truncated

env.close()

## Representación de la Política $\pi$

Para implementar los algoritmos de programación dinámica, necesitamos definir cómo representaremos la política $\pi(s, a)$. 

### Estructura de la Política
La política se almacenará en un **diccionario** de la forma:

```python
pi = {
    ((x, y), a): probabilidad
}
```

Donde:
- $(x, y)$ representa el estado en la grilla.
- $a$ representa una de las cuatro acciones posibles.
- `probabilidad` es la probabilidad de tomar la acción $a$ en el estado $(x, y)$.

Función auxiliar para saber si estamos en un estado final

In [82]:
def is_done(state):
    return state in [(0, 0), (GRID_SIZE - 1, GRID_SIZE - 1)]

### Ejemplo: Política Uniforme
Una política uniforme en la que todas las acciones tienen la misma probabilidad ($ 25\% $) se define como:

In [83]:
pi_rand = {}
for x in range(GRID_SIZE):
    for y in range(GRID_SIZE):
        for a in range(4):  # Cuatro acciones posibles
            if is_done((x, y)): # Si no estamos en un estado terminal
                continue
            pi_rand[((x, y), a)] = 1 / 4  # Probabilidad uniforme

print(pi_rand)

{((0, 1), 0): 0.25, ((0, 1), 1): 0.25, ((0, 1), 2): 0.25, ((0, 1), 3): 0.25, ((0, 2), 0): 0.25, ((0, 2), 1): 0.25, ((0, 2), 2): 0.25, ((0, 2), 3): 0.25, ((0, 3), 0): 0.25, ((0, 3), 1): 0.25, ((0, 3), 2): 0.25, ((0, 3), 3): 0.25, ((1, 0), 0): 0.25, ((1, 0), 1): 0.25, ((1, 0), 2): 0.25, ((1, 0), 3): 0.25, ((1, 1), 0): 0.25, ((1, 1), 1): 0.25, ((1, 1), 2): 0.25, ((1, 1), 3): 0.25, ((1, 2), 0): 0.25, ((1, 2), 1): 0.25, ((1, 2), 2): 0.25, ((1, 2), 3): 0.25, ((1, 3), 0): 0.25, ((1, 3), 1): 0.25, ((1, 3), 2): 0.25, ((1, 3), 3): 0.25, ((2, 0), 0): 0.25, ((2, 0), 1): 0.25, ((2, 0), 2): 0.25, ((2, 0), 3): 0.25, ((2, 1), 0): 0.25, ((2, 1), 1): 0.25, ((2, 1), 2): 0.25, ((2, 1), 3): 0.25, ((2, 2), 0): 0.25, ((2, 2), 1): 0.25, ((2, 2), 2): 0.25, ((2, 2), 3): 0.25, ((2, 3), 0): 0.25, ((2, 3), 1): 0.25, ((2, 3), 2): 0.25, ((2, 3), 3): 0.25, ((3, 0), 0): 0.25, ((3, 0), 1): 0.25, ((3, 0), 2): 0.25, ((3, 0), 3): 0.25, ((3, 1), 0): 0.25, ((3, 1), 1): 0.25, ((3, 1), 2): 0.25, ((3, 1), 3): 0.25, ((3, 2), 0)

### Ejemplo: Política Determinista (derecha y abajo)
Si queremos que el agente siempre elija moverse a la derecha (acción `0`) a excepción de la ultima columna que se mueve hacia abajo (accion `3`), podemos definir la política de la siguiente manera:

In [84]:
pi_der_aba = {}
for x in range(GRID_SIZE):
    for y in range(GRID_SIZE):
        if is_done((x, y)): # Si no estamos en un estado terminal
            continue
        if x == GRID_SIZE - 1:  # Si estamos en la última columna, solo podemos movernos hacia abajo
            pi_der_aba[((x, y), 3)] = 1
        else:  # En otro caso, podemos movernos hacia la derecha
            pi_der_aba[((x, y), 0)] = 1.0
print(pi_der_aba)

{((0, 1), 0): 1.0, ((0, 2), 0): 1.0, ((0, 3), 0): 1.0, ((1, 0), 0): 1.0, ((1, 1), 0): 1.0, ((1, 2), 0): 1.0, ((1, 3), 0): 1.0, ((2, 0), 0): 1.0, ((2, 1), 0): 1.0, ((2, 2), 0): 1.0, ((2, 3), 0): 1.0, ((3, 0), 3): 1, ((3, 1), 3): 1, ((3, 2), 3): 1}


### Ejemplo: Política Determinista (siempre abajo)
Si queremos que el agente siempre elija moverse hacia abajo (acción `3`), podemos definir la política de la siguiente manera:

In [85]:
pi_abajo = {}
for x in range(GRID_SIZE):
    for y in range(GRID_SIZE):
        if is_done((x, y)): # Si no estamos en un estado terminal
                continue
        pi_abajo[((x, y), 3)] = 1



In [86]:
[pi_abajo.get(((2,2), a), 0) for a in range(4)]

[0, 0, 0, 1]

In [87]:
[pi_rand.get(((2,2), a), 0) for a in range(4)]

[0.25, 0.25, 0.25, 0.25]

## Elección de una acción dado un estado y una política
Para elegir una acción dado un estado y una política, necesitamos muestrear una acción de acuerdo a la distribución de probabilidad definida en la política. Para esto, podemos utilizar la función `np.random.choice` de NumPy, que nos permite muestrear un elemento de un conjunto dado con una probabilidad dada.

Más información sobre `np.random.choice` en la [documentación oficial de NumPy](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html).

In [88]:
def select_action(state, policy):
    """
    Dado un estado y una política, devuelve la acción a tomar.

    Parámetros:
    - state: Tupla (x, y) representando el estado actual.
    - policy: Diccionario {((x, y), a): probabilidad} con la política.

    Retorna:
    - La acción seleccionada según la política.
    
    Nota: Si la política es estocástica, se selecciona una acción al azar según las probabilidades.
    recomienda npn random choice
    """
    actions = range(4)
    prob = [policy.get((state, a), 0) for a in actions]
    return np.random.choice(actions, p=prob).item()

## Dinámica del Ambiente ($ p $) 

**La dinámica del ambiente $ p(s' | s, a) $**, que define las transiciones entre estados al tomar una acción. Esta información nos permite conocer, para cada estado y acción, cuál será la probabilidad de llegar al próximo estado y la recompensa recibida.

> Cuando creamos un ambiente con `gym.make()`, Gymnasium aplica un envoltorio (`OrderEnforcing`) que restringe el acceso directo a ciertos atributos internos. Para acceder a `p`, necesitamos usar `env.unwrapped`, que nos da acceso al objeto subyacente sin restricciones.

Ejemplo de acceso a la política desde el ambiente:


In [89]:
p = env.unwrapped.p
print(p)

{((0, 0), 0): [(1.0, (0, 0), 0.0)], ((0, 0), 1): [(1.0, (0, 0), 0.0)], ((0, 0), 2): [(1.0, (0, 0), 0.0)], ((0, 0), 3): [(1.0, (0, 0), 0.0)], ((0, 1), 0): [(1.0, (1, 1), -1.0)], ((0, 1), 1): [(1.0, (0, 0), -1.0)], ((0, 1), 2): [(1.0, (0, 1), -1.0)], ((0, 1), 3): [(1.0, (0, 2), -1.0)], ((0, 2), 0): [(1.0, (1, 2), -1.0)], ((0, 2), 1): [(1.0, (0, 1), -1.0)], ((0, 2), 2): [(1.0, (0, 2), -1.0)], ((0, 2), 3): [(1.0, (0, 3), -1.0)], ((0, 3), 0): [(1.0, (1, 3), -1.0)], ((0, 3), 1): [(1.0, (0, 2), -1.0)], ((0, 3), 2): [(1.0, (0, 3), -1.0)], ((0, 3), 3): [(1.0, (0, 3), -1.0)], ((1, 0), 0): [(1.0, (2, 0), -1.0)], ((1, 0), 1): [(1.0, (1, 0), -1.0)], ((1, 0), 2): [(1.0, (0, 0), -1.0)], ((1, 0), 3): [(1.0, (1, 1), -1.0)], ((1, 1), 0): [(1.0, (2, 1), -1.0)], ((1, 1), 1): [(1.0, (1, 0), -1.0)], ((1, 1), 2): [(1.0, (0, 1), -1.0)], ((1, 1), 3): [(1.0, (1, 2), -1.0)], ((1, 2), 0): [(1.0, (2, 2), -1.0)], ((1, 2), 1): [(1.0, (1, 1), -1.0)], ((1, 2), 2): [(1.0, (0, 2), -1.0)], ((1, 2), 3): [(1.0, (1, 3), -1.

El diccionario tiene la siguiente forma:

```python
self.p = {
    ((x, y), a): [(probabilidad, (nuevo_x, nuevo_y), recompensa)]
}
```

Donde:
- `((x, y), a)`: Clave que representa el **estado actual** `(x, y)` y la **acción** `a` tomada.
- **Valor asociado**: Una lista con **una o más transiciones posibles** en el formato:
  - `probabilidad`: La probabilidad de que ocurra la transición (en este caso, siempre `1.0`, porque el ambiente es determinista).
  - `(nuevo_x, nuevo_y)`: El nuevo estado después de tomar la acción.
  - `recompensa`: El valor de recompensa por realizar la acción.

**Característica de la Política en este Caso**
- **Es determinista**, lo que significa que en cada estado hay una acción con probabilidad 1.
- **En los estados terminales, no importa la acción que se tome, simpre se permanece en el mismo estado.**

> ¿ Cómo podemos ver en el código estas propiedades ?

## Policy Iteration

La **Iteración de Política** (*Policy Iteration*) es un método fundamental en **Programación Dinámica** para encontrar la política óptima $\pi^*$ en un proceso de decisión de Markov (*MDP*) finito. Se basa en dos pasos clave que se repiten iterativamente hasta la convergencia:

1. **Evaluación de Política** (*Policy Evaluation*): Se calcula el valor de la política actual $\pi$, es decir, se obtiene la función de valor $V_\pi(s)$ resolviendo la ecuación de Bellman para todos los estados:
   $$
   V_\pi(s) = \sum_{a} \pi(a | s) \sum_{s', r} p(s', r | s, a) \left[ r + \gamma V_\pi(s') \right]
   $$
   donde $p(s', r | s, a)$ representa la dinámica del ambiente y $\gamma$ es el factor de descuento.

2. **Mejora de Política** (*Policy Improvement*): Se actualiza la política eligiendo en cada estado la acción que maximiza el valor esperado según la función de acción-valor $Q_\pi(s, a)$:
   $$
   \pi'(s) = \arg\max_{a} \sum_{s', r} p(s', r | s, a) \left[ r + \gamma V_\pi(s') \right]
   $$
   Si la nueva política $\pi'$ es diferente de la anterior, se repite el proceso con la nueva política. Si no cambia, hemos encontrado la política óptima $\pi^*$.

Proceso Iterativo:
1. Inicializar una política arbitraria $\pi$.
2. Aplicar **Policy Evaluation** para calcular $V_\pi$.
3. Aplicar **Policy Improvement** para obtener una nueva política $\pi'$.
4. Si $\pi' = \pi$, detenerse. De lo contrario, repetir desde el paso 2.

Este algoritmo garantiza la convergencia a la política óptima en un número finito de iteraciones en ambientes con estados y acciones finitas.

### Implementación de Policy evaluation

$$
\begin{aligned}
\textbf{Input:} \quad & \pi,\ \text{la política a evaluar.} \\
\textbf{Parámetro:} \quad & \theta > 0,\ \text{umbral pequeño que determina la precisión.} \\
\textbf{Inicializar:} \quad & V(s) \text{ arbitrariamente para todo } s \in S^+, \text{ excepto } V(\text{terminal}) = 0. \\
\\
\textbf{Repetir:} \quad & \Delta \gets 0 \\
& \text{Para cada } s \in S: \\
& \quad\quad v \gets V(s) \\
& \quad\quad V(s) \gets \sum_{a} \pi(a \mid s) \sum_{s',r} p(s',r \mid s,a)\,\Bigl[r + \gamma V(s')\Bigr] \\
& \quad\quad \Delta \gets \max\Bigl(\Delta,\, \bigl|v - V(s)\bigr|\Bigr) \\
\\
\textbf{Hasta que:} \quad & \Delta < \theta
\end{aligned}
$$



In [90]:
def policy_evaluation(p, pi, gamma=GAMMA, theta=1e-5, size=GRID_SIZE):
    """
    Evalúa la política 'pi' en el entorno 'env' usando el método de evaluación iterativa.
    
    Parámetros:
        p: dinámica del entorno, diccionario que mapea (estado, acción) a una lista de (probabilidad, estado, recompensa).
        pi: politica a evaluar, diccionario que mapea (estado, acción) a su probabilidad en la política.
        gamma: factor de descuento.
        theta: umbral de convergencia.
    
    Retorna:
        V: función de valor de la política 'pi', diccionario que mapea estado (x, y) a su valor.
    """
    V = {}
    for y in range(size):
        for x in range(size):
            s = (x,y)
            V[s] = 0
    while True:
        delta = 0
        for y in range(size):
            for x in range(size):
                s = (x,y)
                v = V[s]
                V[s] = sum([pi.get((s,a), 0) * prob * (r + gamma * V[s_next]) 
                            for a in range(4) 
                            for (prob, s_next, r) in p[(s,a)]])
                delta = max(delta, abs (v - V[s]))
        if delta < theta:
            break
    return V

In [91]:
V_pi_random = policy_evaluation(p, pi_rand)
print_state_values(V_pi_random, GRID_SIZE)

    0.00   -14.00   -20.00   -22.00
  -14.00   -18.00   -20.00   -20.00
  -20.00   -20.00   -18.00   -14.00
  -22.00   -20.00   -14.00     0.00


In [92]:
V_da_pi = policy_evaluation(p, pi_der_aba)
print_state_values(V_da_pi, GRID_SIZE)

    0.00    -5.00    -4.00    -3.00
   -5.00    -4.00    -3.00    -2.00
   -4.00    -3.00    -2.00    -1.00
   -3.00    -2.00    -1.00     0.00


In [93]:
V_d_pi = policy_evaluation(p, pi_abajo, gamma=0.99)
print_state_values(V_d_pi, GRID_SIZE)

    0.00  -100.00  -100.00    -2.97
 -100.00  -100.00  -100.00    -1.99
 -100.00  -100.00  -100.00    -1.00
 -100.00  -100.00  -100.00     0.00


### Implementación de Policy Improvement

$$
\begin{aligned}
\textbf{Input:} \quad & V,\ \text{la función de valor evaluada}, \\
& p,\ \text{la dinamica del ambiente } \\
& \gamma,\ \text{el factor de descuento.} \\
\\
\textbf{Inicializar:} \quad & \text{Para cada } s \in S \text{ y } a \in A(s),\ new\_pi(s,a) \text{ se asigna arbitrariamente.} \\
\\
\textbf{Para cada } s \in S \text{ (excepto estados terminales):} \quad & \\
& \quad \text{Para cada acción } a \in A(s): \\
& \quad\quad Q(s,a) \gets \sum_{s',r} p(s',r \mid s,a) \Bigl[ r + \gamma\, V(s') \Bigr] \\
& \quad \text{Definir } best\_actions = \{ a \in A(s) \mid Q(s,a) = \max_{a' \in A(s)} Q(s,a') \} \\
& \quad \text{Para cada } a \in A(s): \\
& \quad\quad new\_pi(s,a) \gets 
\begin{cases}
\displaystyle \frac{1}{\lvert best\_actions \rvert}, & \text{si } a \in best\_actions, \\
0, & \text{si } a \notin best\_actions.
\end{cases} \\
\\
\textbf{Retornar:} \quad & new\_pi.
\end{aligned}
$$


In [94]:
def calculate_Q(p, V, gamma=GAMMA, size=GRID_SIZE):
    """
    Calcula el valor Q(s, a) para cada estado y acción.
    
    Parámetros:
        p: dinámica del entorno, diccionario que mapea (estado, acción) a una lista de (probabilidad, estado, recompensa).
        V: función de valor, diccionario que mapea estado (x, y) a su valor.
        gamma: factor de descuento.
    
    Retorna:
        Q: valor Q(s, a).
    """
    Q = {}

    for y in range(size):
        for x in range(size):
            s = (x,y)
            if not is_done(s):
                for a in range(4):
                    Q[(s,a)] = sum(prob * (r + gamma * V[s_next]) 
                                for (prob, s_next, r) in p[(s,a)])
            else:
                for a in range(4):
                    Q[(s,a)] = 0
    return Q
    

In [95]:
Q_pi_random = calculate_Q(p, V_pi_random)
print_state_action_values(Q_pi_random, GRID_SIZE)

Valores para la acción Derecha (acción 0):
    0.00   -21.00   -23.00   -23.00
  -19.00   -21.00   -21.00   -21.00
  -21.00   -19.00   -15.00   -15.00
  -21.00   -15.00    -1.00     0.00


Valores para la acción Arriba (acción 1):
    0.00   -15.00   -21.00   -23.00
   -1.00   -15.00   -21.00   -23.00
  -15.00   -19.00   -21.00   -21.00
  -21.00   -21.00   -19.00     0.00


Valores para la acción Izquierda (acción 2):
    0.00    -1.00   -15.00   -21.00
  -15.00   -15.00   -19.00   -21.00
  -21.00   -21.00   -21.00   -19.00
  -23.00   -23.00   -21.00     0.00


Valores para la acción Abajo (acción 3):
    0.00   -19.00   -21.00   -21.00
  -21.00   -21.00   -19.00   -15.00
  -23.00   -21.00   -15.00    -1.00
  -23.00   -21.00   -15.00     0.00




In [96]:
Q_da_pi = calculate_Q(p, V_da_pi)
print_state_action_values(Q_da_pi, GRID_SIZE)

Valores para la acción Derecha (acción 0):
    0.00    -5.00    -4.00    -4.00
   -5.00    -4.00    -3.00    -3.00
   -4.00    -3.00    -2.00    -2.00
   -3.00    -2.00    -1.00     0.00


Valores para la acción Arriba (acción 1):
    0.00    -6.00    -5.00    -4.00
   -1.00    -6.00    -5.00    -4.00
   -6.00    -5.00    -4.00    -3.00
   -5.00    -4.00    -3.00     0.00


Valores para la acción Izquierda (acción 2):
    0.00    -1.00    -6.00    -5.00
   -6.00    -6.00    -5.00    -4.00
   -5.00    -5.00    -4.00    -3.00
   -4.00    -4.00    -3.00     0.00


Valores para la acción Abajo (acción 3):
    0.00    -5.00    -4.00    -3.00
   -5.00    -4.00    -3.00    -2.00
   -4.00    -3.00    -2.00    -1.00
   -4.00    -3.00    -2.00     0.00




In [97]:
Q_d_pi = calculate_Q(p, V_d_pi, gamma=0.99)
print_state_action_values(Q_d_pi, GRID_SIZE)

Valores para la acción Derecha (acción 0):
    0.00  -100.00    -3.94    -3.94
 -100.00  -100.00    -2.97    -2.97
 -100.00  -100.00    -1.99    -1.99
 -100.00  -100.00    -1.00     0.00


Valores para la acción Arriba (acción 1):
    0.00  -100.00  -100.00    -3.94
   -1.00  -100.00  -100.00    -3.94
 -100.00  -100.00  -100.00    -2.97
 -100.00  -100.00  -100.00     0.00


Valores para la acción Izquierda (acción 2):
    0.00    -1.00  -100.00  -100.00
 -100.00  -100.00  -100.00  -100.00
 -100.00  -100.00  -100.00  -100.00
 -100.00  -100.00  -100.00     0.00


Valores para la acción Abajo (acción 3):
    0.00  -100.00  -100.00    -2.97
 -100.00  -100.00  -100.00    -1.99
 -100.00  -100.00  -100.00    -1.00
 -100.00  -100.00  -100.00     0.00




In [98]:
def improve_policy(p, V, gamma=GAMMA, size=GRID_SIZE):
    """
    Genera una política mejorada (greedy) a partir de la función de valor V.
    
    Parámetros:
        env: entorno que posee un atributo 'p' con la dinámica.
        V: diccionario que mapea estados a sus valores.
        gamma: factor de descuento.
    
    Retorna:
        new_pi: diccionario que representa la política mejorada, mapeando (estado, acción) a probabilidad.
    """
    new_pi = {}
    Q = calculate_Q(p, V, gamma)
    tol = 1e-4

    for y in range(size):
        for x in range(size):
            s = (x,y)
            if is_done(s):
                continue

            mejores_a = [0]
            mejor_q = Q[(s,0)]
            for a in range(1,4): 
                q_dif = Q[(s,a)] - mejor_q
                if q_dif > tol:
                    mejor_q = Q[(s,a)]
                    mejores_a = [a]
                elif (tol > q_dif or tol == q_dif) and (q_dif > (-tol) or q_dif == (-tol)):
                    mejores_a.append(a)

            for a in range(4):
                new_pi[(s,a)] = 1 / len(mejores_a) if a in mejores_a else 0

    return new_pi

In [99]:
V_pi_random = policy_evaluation(p, pi_rand)
print_state_values(V_pi_random, GRID_SIZE)


    0.00   -14.00   -20.00   -22.00
  -14.00   -18.00   -20.00   -20.00
  -20.00   -20.00   -18.00   -14.00
  -22.00   -20.00   -14.00     0.00


In [100]:
pi_mejorada = improve_policy(p, V_pi_random)
V_mejorada = policy_evaluation(p, pi_mejorada)
print_state_values(V_mejorada, GRID_SIZE)

    0.00    -1.00    -2.00    -3.00
   -1.00    -2.00    -3.00    -2.00
   -2.00    -3.00    -2.00    -1.00
   -3.00    -2.00    -1.00     0.00


In [101]:
print_policy_grids(pi_mejorada, GRID_SIZE)

Política para la acción Derecha (acción 0):
  0.00   0.00   0.00   0.00
  0.00   0.00   0.00   0.00
  0.00   0.50   0.50   0.00
  0.50   1.00   1.00   0.00


Política para la acción Arriba (acción 1):
  0.00   0.00   0.00   0.00
  1.00   0.50   0.00   0.00
  1.00   0.50   0.00   0.00
  0.50   0.00   0.00   0.00


Política para la acción Izquierda (acción 2):
  0.00   1.00   1.00   0.50
  0.00   0.50   0.50   0.00
  0.00   0.00   0.00   0.00
  0.00   0.00   0.00   0.00


Política para la acción Abajo (acción 3):
  0.00   0.00   0.00   0.50
  0.00   0.00   0.50   1.00
  0.00   0.00   0.50   1.00
  0.00   0.00   0.00   0.00




In [102]:
def compare_policies(pi_1, pi_2, tol, size=GRID_SIZE):
    """
    Compara dos políticas y retorna True si son iguales (considerando una tolerancia en los valores), o False en caso contrario.
    
    Parámetros:
        pi_1, pi_2: diccionarios que representan las políticas (mapean (estado, acción) a probabilidad).
        tol: tolerancia para comparar las probabilidades.
    
    Retorna:
        True si ambas políticas son iguales en todos los (estado, acción); False de lo contrario.
    """
    delta = 0
    for y in range(size):
        for x in range(size):
            s = (x,y)
            for a in range(4):
                if is_done(s):
                    continue
                distr_1 = pi_1.get(((s), a), 0)
                distr_2 = pi_2.get(((s), a), 0)
                delta = max(delta, abs(distr_1 - distr_2))
    return delta < tol

In [103]:
def policy_iteration(p, pi_init, gamma, theta, size=GRID_SIZE):
    """
    Realiza el algoritmo de iteración de política.
    
    Parámetros:
        p: dinámica del entorno, diccionario que mapea (estado, acción) a una lista de (probabilidad, estado, recompensa).
        gamma: factor de descuento.
        theta: umbral de convergencia.
    
    Retorna:
        pi: política óptima.
        iter: cantidad de iteraciones necesarias para converger

    pi_int la mejoramos
    la evaluamos
    si no mejora
    iteramos
    """
    iter = 0
    while True:
        iter += 1
        V = policy_evaluation(p, pi_init)
        pi = improve_policy(p, V)
        if compare_policies(pi_init, pi, tol=theta):
            break
        else:
            pi_init = pi
    return pi, iter

In [104]:
pi_star, iter = policy_iteration(p, pi_rand, gamma=GAMMA, theta=1e-6)
print(iter)
print_policy_grids(pi_star, GRID_SIZE)

3
Política para la acción Derecha (acción 0):
  0.00   0.00   0.00   0.00
  0.00   0.00   0.25   0.00
  0.00   0.25   0.50   0.00
  0.50   1.00   1.00   0.00


Política para la acción Arriba (acción 1):
  0.00   0.00   0.00   0.00
  1.00   0.50   0.25   0.00
  1.00   0.25   0.00   0.00
  0.50   0.00   0.00   0.00


Política para la acción Izquierda (acción 2):
  0.00   1.00   1.00   0.50
  0.00   0.50   0.25   0.00
  0.00   0.25   0.00   0.00
  0.00   0.00   0.00   0.00


Política para la acción Abajo (acción 3):
  0.00   0.00   0.00   0.50
  0.00   0.00   0.25   1.00
  0.00   0.25   0.50   1.00
  0.00   0.00   0.00   0.00




> 1) Explicar por qué esta política óptima tiene dicha distribución de probabilidad. 
> 2) Existen otras políticas óptimas posibles ?

#### Respuesta:

1: 
La distribución de la política se puede explicar de manera intuitiva. Para los estados adyacentes a las casillas terminales, como por ejemplo (0,1) utilizando la notación (fila, col), la distribución de la política es (0, 0, 1, 0), porque la casilla terminal (0,0) está a un movimiento de distancia.

Por lo tanto, en este ejemplo, movernos a la izquierda maximiza nuestra recompensa. Lo mismo sucede para todas las casillas que comparten o una fila o una columna con las casillas terminales; el movimiento más corto implica repetir la acción con dirección a la casilla terminal con la que comparte fila o columna.

Sin embargo, hay casos interesantes como las casillas (0,3) y (3,0). La casilla (0,3) comparte la fila con una terminal y la columna con otra, y viceversa para (3,0). En estas situaciones, moverse a lo largo de la fila compartida hacia su terminal o moverse a lo largo de la columna compartida hacia la otra terminal son ambas opciones igualmente óptimas en términos de distancia. Por esta razón, la distribución de la política para la casilla (1,1) es (0, 0.5, 0.5, 0), indicando una probabilidad de 0.5 para cada una de las dos acciones óptimas.

Por último, en las casillas con fila entre {1, 2} y columna entre {1, 2}, que no comparten ninguna fila o columna con las casillas terminales. Se da que hay dos o incluso cuatro caminos óptimos posibles. Por eso vemos una distribución de 0.5 y 0.25 para las acciones que resultan en esos caminos óptimos.

2:
Como ya indiqué, pueden existir diversas rutas óptimas. 

Una política podría limitarse a solo uno de los caminos. Pero como son igualmente óptimos, generaría la misma recompensa que otra política que explore ambos con una distribución equitativa (0.5 cada uno). 

Así como también sería óptima una política que toma un camino óptimo el 0.3 y 0.7 el otro. Fundamentalmente, múltiples políticas pueden optimizar la recompensa.

In [105]:
V_pi_star = policy_evaluation(p, pi_star)
print_state_values(V_pi_star, GRID_SIZE)

    0.00    -1.00    -2.00    -3.00
   -1.00    -2.00    -3.00    -2.00
   -2.00    -3.00    -2.00    -1.00
   -3.00    -2.00    -1.00     0.00


In [106]:
Q_pi_star = calculate_Q(p, V_pi_star)
print_state_action_values(Q_pi_star, GRID_SIZE)

Valores para la acción Derecha (acción 0):
    0.00    -3.00    -4.00    -4.00
   -3.00    -4.00    -3.00    -3.00
   -4.00    -3.00    -2.00    -2.00
   -3.00    -2.00    -1.00     0.00


Valores para la acción Arriba (acción 1):
    0.00    -2.00    -3.00    -4.00
   -1.00    -2.00    -3.00    -4.00
   -2.00    -3.00    -4.00    -3.00
   -3.00    -4.00    -3.00     0.00


Valores para la acción Izquierda (acción 2):
    0.00    -1.00    -2.00    -3.00
   -2.00    -2.00    -3.00    -4.00
   -3.00    -3.00    -4.00    -3.00
   -4.00    -4.00    -3.00     0.00


Valores para la acción Abajo (acción 3):
    0.00    -3.00    -4.00    -3.00
   -3.00    -4.00    -3.00    -2.00
   -4.00    -3.00    -2.00    -1.00
   -4.00    -3.00    -2.00     0.00




In [107]:
env = get_environment()

for episode_num in range(10):
    obs, info = env.reset()

    episode_over = False
    while not episode_over:
        state = (obs["pos"][0], obs["pos"][1])
        action = select_action(state, pi_star)
        obs, reward, terminated, truncated, info = env.step(action)
        episode_over = terminated or truncated

env.close()

### Pruebas con distintas políticas iniciales

#### Prueba con política determinística

In [108]:
pi_star_2, iter_2 = policy_iteration(p, pi_der_aba, gamma=1, theta=1e-6)
print(iter_2)
print_policy_grids(pi_star_2, GRID_SIZE)

4
Política para la acción Derecha (acción 0):
  0.00   0.00   0.00   0.00
  0.00   0.00   0.25   0.00
  0.00   0.25   0.50   0.00
  0.50   1.00   1.00   0.00


Política para la acción Arriba (acción 1):
  0.00   0.00   0.00   0.00
  1.00   0.50   0.25   0.00
  1.00   0.25   0.00   0.00
  0.50   0.00   0.00   0.00


Política para la acción Izquierda (acción 2):
  0.00   1.00   1.00   0.50
  0.00   0.50   0.25   0.00
  0.00   0.25   0.00   0.00
  0.00   0.00   0.00   0.00


Política para la acción Abajo (acción 3):
  0.00   0.00   0.00   0.50
  0.00   0.00   0.25   1.00
  0.00   0.25   0.50   1.00
  0.00   0.00   0.00   0.00




#### Prueba con política estocástica

In [109]:
pi_arr_der_izq = {}
for x in range(GRID_SIZE):
    for y in range(GRID_SIZE):
        if is_done((x, y)): # Si no estamos en un estado terminal
                continue
        pi_arr_der_izq[((x, y), 0)] = 0.8
        pi_arr_der_izq[((x, y), 1)] = 0.1
        pi_arr_der_izq[((x, y), 2)] = 0.1
        pi_arr_der_izq[((x, y), 3)] = 0
print(pi_arr_der_izq)

{((0, 1), 0): 0.8, ((0, 1), 1): 0.1, ((0, 1), 2): 0.1, ((0, 1), 3): 0, ((0, 2), 0): 0.8, ((0, 2), 1): 0.1, ((0, 2), 2): 0.1, ((0, 2), 3): 0, ((0, 3), 0): 0.8, ((0, 3), 1): 0.1, ((0, 3), 2): 0.1, ((0, 3), 3): 0, ((1, 0), 0): 0.8, ((1, 0), 1): 0.1, ((1, 0), 2): 0.1, ((1, 0), 3): 0, ((1, 1), 0): 0.8, ((1, 1), 1): 0.1, ((1, 1), 2): 0.1, ((1, 1), 3): 0, ((1, 2), 0): 0.8, ((1, 2), 1): 0.1, ((1, 2), 2): 0.1, ((1, 2), 3): 0, ((1, 3), 0): 0.8, ((1, 3), 1): 0.1, ((1, 3), 2): 0.1, ((1, 3), 3): 0, ((2, 0), 0): 0.8, ((2, 0), 1): 0.1, ((2, 0), 2): 0.1, ((2, 0), 3): 0, ((2, 1), 0): 0.8, ((2, 1), 1): 0.1, ((2, 1), 2): 0.1, ((2, 1), 3): 0, ((2, 2), 0): 0.8, ((2, 2), 1): 0.1, ((2, 2), 2): 0.1, ((2, 2), 3): 0, ((2, 3), 0): 0.8, ((2, 3), 1): 0.1, ((2, 3), 2): 0.1, ((2, 3), 3): 0, ((3, 0), 0): 0.8, ((3, 0), 1): 0.1, ((3, 0), 2): 0.1, ((3, 0), 3): 0, ((3, 1), 0): 0.8, ((3, 1), 1): 0.1, ((3, 1), 2): 0.1, ((3, 1), 3): 0, ((3, 2), 0): 0.8, ((3, 2), 1): 0.1, ((3, 2), 2): 0.1, ((3, 2), 3): 0}


In [110]:
pi_star_3, iter_3 = policy_iteration(p, pi_arr_der_izq, gamma=1, theta=1e-6)
print(iter_3)
print_policy_grids(pi_star_3, GRID_SIZE)

4
Política para la acción Derecha (acción 0):
  0.00   0.00   0.00   0.00
  0.00   0.00   0.25   0.00
  0.00   0.25   0.50   0.00
  0.50   1.00   1.00   0.00


Política para la acción Arriba (acción 1):
  0.00   0.00   0.00   0.00
  1.00   0.50   0.25   0.00
  1.00   0.25   0.00   0.00
  0.50   0.00   0.00   0.00


Política para la acción Izquierda (acción 2):
  0.00   1.00   1.00   0.50
  0.00   0.50   0.25   0.00
  0.00   0.25   0.00   0.00
  0.00   0.00   0.00   0.00


Política para la acción Abajo (acción 3):
  0.00   0.00   0.00   0.50
  0.00   0.00   0.25   1.00
  0.00   0.25   0.50   1.00
  0.00   0.00   0.00   0.00




#### Conclusión

Se observa que toda política inicial escogida termina derivando la misma política óptima. 

La política aleatoria converge más rápido, necesitando solo 3 iteraciones frente a las 4 de las otras. 

Esto sugiere que su inicio con igual probabilidad para cada acción se parece más a la política óptima.

## Value Iteration

Value Iteration es un algoritmo de programación dinámica empleado para estimar la función de valor óptima $ V^* $ y, a partir de ella, derivar la política óptima $ \pi^* $. Este método se basa en actualizar iterativamente los valores de cada estado utilizando la ecuación de Bellman, hasta que las actualizaciones sean menores que un pequeño umbral $ \theta $ que determina la precisión de la estimación.

El proceso general es el siguiente:

1. **Inicialización:**  
   Se asigna un valor arbitrario a $ V(s) $ para todos los estados $ s \in S^+ $, excepto en los estados terminales, donde se establece $ V(\text{terminal}) = 0 $.

2. **Actualización Iterativa:**  
   Para cada estado $ s \in S $, se actualiza su valor mediante la fórmula:
   $$
   V(s) \leftarrow \max_{a} \sum_{s', r} p(s', r \mid s, a) \Bigl[ r + \gamma V(s') \Bigr],
   $$
   donde $ p(s', r \mid s, a) $ es la probabilidad de transitar al estado $ s' $ y recibir la recompensa $ r $ al tomar la acción $ a $ en el estado $ s $.  
   La iteración continúa hasta que la diferencia máxima entre los valores antiguos y actualizados en todos los estados sea menor que $ \theta $.

3. **Extracción de la Política:**  
   Una vez convergido $ V $, se define la política óptima determinista $ \pi^* $ de la siguiente manera:
   $$
   \pi^*(s) = \operatorname*{argmax}_{a} \sum_{s', r} p(s', r \mid s, a) \Bigl[ r + \gamma V(s') \Bigr].
   $$

Este algoritmo es fundamental en el aprendizaje por refuerzo y en la resolución de procesos de decisión de Markov, ya que permite obtener de manera eficiente tanto la función de valor óptima como la política que maximiza el retorno esperado en cada estado.


In [111]:
def max_dif_dict(dict_1, dict_2):
    ''''
    Para dos diccionarios con claves idénticas y no vacíos. Devuelve la mayor diferencia uno a uno entre los dos diccionarios.
    Parámetros:
        dict_1: primer diccionario
        dict_2: segundo diccionario

    Retorna:
        max_dif: máxima diferencia entre los mismos
    '''
    max_dif = 0
    for x in dict_1: 
        val_1 = dict_1[x]
        val_2 = dict_2[x]
        dif = val_1 - val_2
        if dif > max_dif:
            max_dif = dif

    return max_dif


In [112]:
def value_iteration(p, gamma=GAMMA, theta=1e-5, size=GRID_SIZE):
    """
    Realiza el algoritmo de iteración de valor.
    
    Parámetros:
        p: dinámica del entorno, diccionario que mapea (estado, acción) a una lista de (probabilidad, estado, recompensa).
        gamma: factor de descuento.
        theta: umbral de convergencia.
    
    Retorna:
        pi: la pólitica óptima.
    """
    V_init = {}
    # Inicializo V con valores arbitrarios
    for y in range(size):
        for x in range(size):
            s = (x,y)
            if is_done(s):
                V_init[s] = 0
            else:
                V_init[s] = -6

    V = V_init.copy()
    # Iteración para actualizar el valor de V
    while True:
        for y in range(4):
            for x in range(4):
                s = (x,y)
                if is_done(s):
                    continue
                mejor_v = -np.inf
                for a in range(4):
                    new_v = sum(prob * (r + gamma * V[s_next]) 
                                for (prob, s_next, r) in p[(s,a)])
                    if new_v > mejor_v:
                        mejor_v = new_v
                V[s] = mejor_v
        # Evaluó máxima diferencia entre valores actualizados y antiguos
        max_dif = max_dif_dict(V, V_init)
        if theta > max_dif:
            break
        else:
            V_init = V

    pi = {}
    # Extraigo la mejor política
    for y in range(size):
        for x in range(size):
            s = (x, y)
            if is_done(s):
                pi[s] = 0
                continue
            a_elegida = []
            mejor_valor_a = -np.inf
            for a in range(4):
                valor_a = 0
                for prob, s_next, r in p[(s, a)]:
                    valor_a += prob * (r + gamma * V[s_next])
                if valor_a > mejor_valor_a:
                    mejor_valor_a = valor_a
                    a_elegida = [a]
                elif valor_a == mejor_valor_a:
                    a_elegida.append(a)
            for a in range(4):
                pi[(s,a)] = 1 / len(a_elegida) if a in a_elegida else 0 

    return pi

In [113]:
pi_star_4 = value_iteration(p)
print_policy_grids(pi_star_4, GRID_SIZE)

Política para la acción Derecha (acción 0):
  0.00   0.00   0.00   0.00
  0.00   0.00   0.25   0.00
  0.00   0.25   0.50   0.00
  0.50   1.00   1.00   0.00


Política para la acción Arriba (acción 1):
  0.00   0.00   0.00   0.00
  1.00   0.50   0.25   0.00
  1.00   0.25   0.00   0.00
  0.50   0.00   0.00   0.00


Política para la acción Izquierda (acción 2):
  0.00   1.00   1.00   0.50
  0.00   0.50   0.25   0.00
  0.00   0.25   0.00   0.00
  0.00   0.00   0.00   0.00


Política para la acción Abajo (acción 3):
  0.00   0.00   0.00   0.50
  0.00   0.00   0.25   1.00
  0.00   0.25   0.50   1.00
  0.00   0.00   0.00   0.00




In [114]:
env = get_environment()

for episode_num in range(10):
    obs, info = env.reset()

    episode_over = False
    while not episode_over:
        state = (obs["pos"][0], obs["pos"][1])
        action = select_action(state, pi_star_4)
        obs, reward, terminated, truncated, info = env.step(action)
        episode_over = terminated or truncated

env.close()

## **Tareas**

**1. Comparación entre diferentes políticas iniciales**
- Prueba iniciar la **Iteración de Política** con diferentes políticas iniciales (aleatorias, deterministas, uniformes).
- ¿La política óptima cambia en función de la inicialización?
- ¿Cuántas iteraciones necesita cada caso para converger?
  
**2. [extra] Impacto de la estructura del Grid World**
- Modifica el tamaño de la grilla y analiza su impacto en el desempeño de los algoritmos.

**3. [extra] Evaluación en un entorno estocástico**
- Introduce aleatoriedad en la transición de estados (por ejemplo, que el agente no siempre se mueva exactamente en la dirección deseada). (usar "gymnasium_env/GridWorld_stochastic-v0")
- ¿Cómo cambia la política óptima en este caso?